기능별 collection 생성

In [ ]:
import os
import json
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import chromadb
from glob import glob

#API Key 설정 
os.environ["OPENAI_API_KEY"] = ""

#임베딩 모델 초기화 
embeddings_model = OpenAIEmbeddings(model = 'text-embedding-3-small')

#json-> text 파일을 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

# Chroma 클라이언트
client = chromadb.PersistentClient(path="./chroma_db")

#json -> documents 파일 변환

def load_json_documents(json_files):
    docs = []
    for file in json_files:
        with open(file, "r", encoding="utf-8") as f:
            data = json.load(f)

        #시나리오
        for item in data:
            if "Text" in item and "Completion" in item:
                docs.append(Document(
                    page_content=f"질문: {item['Text']}\n답변: {item['Completion']}",
                    metadata={
                        "카테고리": item.get("카테고리", "")
                    }
                ))

            # 수강이력
            if "lecture_name" in item and "student_id" in item:
                content = (
                    f"학번: {item['student_id']}\n"
                    f"강의명: {item['lecture_name']}\n"
                    f"학정번호: {item['lecture_id']}\n"
                    f"개설 학과: {item['department_offered']}\n"
                    f"이수 구분: {item['lecture_course_type']}\n"
                    f"학점: {item['lecture_credit']}학점\n"
                    f"성적: {item['lecutre_grade']}"
                )
                if "retake_or_delete_status" in item:
                    content += f"\n재수강/삭제 여부: {item['retake_or_delete_status']}"
                if "retake_status" in item:
                    content += f"\n재수강 여부: {item['retake_status']}"

                docs.append(Document(
                    page_content=content,
                    metadata={
                        "lecture_name": item["lecture_name"],
                        "course_type": item["lecture_course_type"]
                    }
                ))

            # 강의 탐색
            if "lecture_id" in item and "lecture_name" in item and "student_id" not in item:
                content = (
                    f"학정번호: {item.get('lecture_id', '')}\n"
                    f"강의명: {item.get('lecture_name', '')}\n"
                    f"강의평점: {item.get('lecture_ratings', '')}\n"
                    f"과제: {item.get('lecture_homework', '')}\n"
                    f"팀플: {item.get('lecture_team', '')}\n"
                    f"성적평가정도: {item.get('lecutre_grade', '')}\n"
                    f"출결 방식: {item.get('lecutre_attendance', '')}\n"
                    f"시험 횟수: {item.get('lecutre_test', '')}\n"
                    f"시험 방식: {item.get('lecture_testinform', '')}\n"
                    f"전공 학점: {item.get('credits_major', '없음')}\n"
                    f"교양 학점: {item.get('credits_general', '없음')}\n"
                    f"총 학점: {item.get('credits_total', '없음')}\n"
                    f"교수명: {item.get('lecture_professorname', '')}\n"
                    f"수업 시간: {item.get('lecture_time', '')}\n"
                    f"강의 유형: {item.get('lecture_course_type', '')}\n"
                    f"학점: {item.get('lecture_hours', '')}시간\n"
                    f"학기: {item.get('lecture_semester', '')}학기\n"
                    f"강의 설명: {item.get('lecture_inform', '')}"
                )
                docs.append(Document(
                    page_content=content,
                    metadata={
                        "lecture_id": item.get("lecture_id", ""),
                        "lecture_name": item.get("lecture_name", ""),
                        "lecture_ratings": item.get("lecture_ratings", ""),
                        "lecture_team": item.get("lecture_team", ""),
                        "lecutre_grade": item.get("lecutre_grade", ""),
                        "professor": item.get("lecture_professorname", "")
                    }
                ))
    return docs


base_path = r"C:\Users\82103\OneDrive\바탕 화면\data"

user_datasets = {
    "kim": glob(os.path.join(base_path, "academic_status", "kim", "*.json")),
    "hong": glob(os.path.join(base_path, "academic_status", "hong", "*.json"))
}


task_datasets = {
    task: glob(os.path.join(base_path, task, "*.json"))
    for task in ["lecture_search", "career_counsel", "academic_status"]
}

# 각 기능별 컬렉션 생성 및 임베딩 처리
for task, files in task_datasets.items():
    # JSON → Documents
    docs = load_json_documents(files)

    #청크 분할
    split_docs = text_splitter.split_documents(docs)

    # 청크 텍스트
    texts = [doc.page_content for doc in split_docs]

    # 임베딩
    embeddings = embeddings_model.embed_documents(texts)

    # 저장
    collection = client.get_or_create_collection(name=task)
    collection.add(
        documents=texts,
        embeddings=embeddings,
        ids=[f"{task}_{i}" for i in range(len(texts))]
    )
    
    print("임베딩 개수:", len(embeddings))
    print("벡터 차원:", len(embeddings[0]))

Insert of existing embedding ID: lecture_search_0
Insert of existing embedding ID: lecture_search_1
Insert of existing embedding ID: lecture_search_2
Insert of existing embedding ID: lecture_search_3
Insert of existing embedding ID: lecture_search_4
Insert of existing embedding ID: lecture_search_5
Insert of existing embedding ID: lecture_search_6
Insert of existing embedding ID: lecture_search_7
Insert of existing embedding ID: lecture_search_8
Insert of existing embedding ID: lecture_search_9
Insert of existing embedding ID: lecture_search_10
Insert of existing embedding ID: lecture_search_11
Insert of existing embedding ID: lecture_search_12
Insert of existing embedding ID: lecture_search_13
Insert of existing embedding ID: lecture_search_14
Insert of existing embedding ID: lecture_search_15
Insert of existing embedding ID: lecture_search_16
Insert of existing embedding ID: lecture_search_17
Insert of existing embedding ID: lecture_search_18
Insert of existing embedding ID: lecture_

임베딩 개수: 609
벡터 차원: 1536


Add of existing embedding ID: career_counsel_0
Add of existing embedding ID: career_counsel_1
Add of existing embedding ID: career_counsel_2
Add of existing embedding ID: career_counsel_3
Add of existing embedding ID: career_counsel_4
Add of existing embedding ID: career_counsel_5
Add of existing embedding ID: career_counsel_6
Add of existing embedding ID: career_counsel_7
Add of existing embedding ID: career_counsel_8
Add of existing embedding ID: career_counsel_9
Add of existing embedding ID: career_counsel_10
Add of existing embedding ID: career_counsel_11
Add of existing embedding ID: career_counsel_12
Add of existing embedding ID: career_counsel_13
Add of existing embedding ID: career_counsel_14
Add of existing embedding ID: career_counsel_15
Add of existing embedding ID: career_counsel_16
Add of existing embedding ID: career_counsel_17
Add of existing embedding ID: career_counsel_18
Add of existing embedding ID: career_counsel_19
Add of existing embedding ID: career_counsel_20
Ad

임베딩 개수: 591
벡터 차원: 1536


Add of existing embedding ID: academic_status_0
Add of existing embedding ID: academic_status_1
Add of existing embedding ID: academic_status_2
Add of existing embedding ID: academic_status_3
Add of existing embedding ID: academic_status_4
Add of existing embedding ID: academic_status_5
Add of existing embedding ID: academic_status_6
Add of existing embedding ID: academic_status_7
Add of existing embedding ID: academic_status_8
Add of existing embedding ID: academic_status_9
Add of existing embedding ID: academic_status_10
Add of existing embedding ID: academic_status_11
Add of existing embedding ID: academic_status_12
Add of existing embedding ID: academic_status_13
Add of existing embedding ID: academic_status_14
Add of existing embedding ID: academic_status_15
Add of existing embedding ID: academic_status_16
Add of existing embedding ID: academic_status_17
Add of existing embedding ID: academic_status_18
Add of existing embedding ID: academic_status_19
Add of existing embedding ID: 

임베딩 개수: 40
벡터 차원: 1536


사용자의 입력을 기능별로 분류
강의 탐색 -> 강의탐색기능 
학습 현황 -> 학습현황기능
진로 상담 -> 진로상담기능 

In [47]:
def classify_function(user_input: str) -> str:
    if "강의" in user_input or "탐색" in user_input:
        return "lecture_search"
    elif "진로" in user_input or "상담" in user_input:
        return "career_counsel"
    elif "학업" in user_input or "현황" in user_input:
        return "academic_status"
    else:
        return "lecture_search"  # fallback

분류된 기능에 맞춰 ChromaDB 검색

In [48]:
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.3)

def answer_query(user_input: str):
    task = classify_function(user_input)

    # 컬렉션 로드
    vectorstore = Chroma(
        persist_directory="./chroma_db",
        collection_name=task,
        embedding_function=embeddings_model
    )

    # 유사 문서 검색
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
    qa = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type="stuff")

    # 답변 생성
    return qa.run(user_input)

In [49]:
def answer_query(user_input: str, user_id: str = "kim"):
    task = classify_function(user_input)
    collection_name = task

    vectorstore = Chroma(
        persist_directory="./chroma_db",
        collection_name=collection_name,
        embedding_function=embeddings_model
    )

    prompt = PromptTemplate(
        input_variables=["context", "question"],
        template= """
        다음은 사용자의 수강 이력 또는 학업/진로 관련 정보입니다.
        질문에 답할 때 반드시 정확한 정보를 바탕으로 하세요.
    
        성적은 A+, A0, B+, B0, C+, C0, F 등의 형식이며, 
        ❗ 절대 유사한 등급을 포함하거나 추론하지 마세요.
        예: "A+"를 요청했을 경우, 반드시 성적이 "A+"인 과목만 포함하세요. "A0"는 포함하면 안 됩니다.
    
        📌 답변 형식 지침:
        - 질문에 해당하는 내용이 **여러 개인 경우 빠짐없이 모두 나열**하세요.
        -과목, 강의 추천의 경우 주어진 데이터 만을 활용하여 답변하세요. 데이터에 없는 과목은 언급하지 마세요.
        -중복되는 단어는 한번만 언급하세요.
        - 질문 유형에 따라 말끝을 다르게 쓰세요.
          예:
            - "내가 A+ 받은 과목 알려줘" → "OOO, OOO 과목에서 A+를 받으셨습니다."
            - "자료구조 성적이 뭐야?" → "자료구조의 성적은 B+입니다."
            - "객체지향프로그래밍이 전선이지?" → "네, 객체지향프로그래밍은 전필입니다."
            - "전체 평균이 어떻게 돼?" → "전체 성적 평균은 3.58입니다."
        - 정중하고 간결하게, 핵심만 자연스럽게 응답하세요.
        - 축하 인사, 추가 질문 권유 등은 하지 마세요.
    
        문서:
        {context}
    
        질문:
        {question}
    
        이 지침을 따라 정확하고 정중하게 답변하세요.
        """
    )

    qa = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=vectorstore.as_retriever(search_kwargs={"k": 20}),
        chain_type="stuff",
        chain_type_kwargs={"prompt": prompt}
    )

    return qa.invoke({"query": user_input})["result"]


In [50]:
answer_query(user_input="내가 A+ 받은 과목 알려줘", user_id="kim")

'- AI빅데이터의이해, AI멀티미디어활용 과목에서 A+를 받으셨습니다.'

In [28]:
query = "팀플 없는 전공 추천해줘 "
print(answer_query(query))

답변: C프로그래밍 강의는 팀플이 없는 강의입니다.


In [38]:
query = "나 재수강 총 몇번 했나? "
print(answer_query(query))

재수강 횟수는 총 4번입니다.
